In [ ]:
import os

import numpy as np
import pandas as pd
import random

from agents.matchmanager import buildMatchManager

from multiprocessing import Pool
from agents.matchmanager import MatchManager

from itertools import product

from agents import GreedyAgent, AlphaBetaAgent, RandomAgent
from scenarios import scenarioJunction, scenarioJunctionExo, scenarioTest1v1, scenarioTest2v2
from core.const import RED, BLUE
from agents.ml.simple import SimpleMLAgent
import json

In [ ]:
def game(args):
    i,s,agentRed,agentBlue=args
    board, state = s()
    if agentRed == SimpleMLAgent and agentBlue == SimpleMLAgent:
        playerRed = agentRed(RED, {'scenario': board.name, 'model': 'RandomForestClassifier'})
        playerBlue = agentBlue(BLUE, {'scenario': board.name, 'model': 'RandomForestClassifier'})
    elif agentRed == SimpleMLAgent and agentBlue != SimpleMLAgent:
        playerRed = agentRed(RED, {'scenario': board.name, 'model': 'RandomForestClassifier'})
        playerBlue = agentBlue(BLUE)
    elif agentBlue == SimpleMLAgent and agentRed != SimpleMLAgent:
        playerBlue = agentBlue(BLUE, {'scenario': board.name, 'model': 'RandomForestClassifier'})
        playerRed = agentRed(RED)
    else:
        playerBlue = agentBlue(BLUE)
        playerRed = agentRed(RED)
    print(playerRed.name)
    print(playerBlue.name)
    print(board.name)
    mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=42 + i)
    while not mm.end:
        mm.nextStep()
    save={
        'scenario':board.name,
        'seed':42+i,
        'teamRed':playerRed.name,
        'teamBlue':playerBlue.name,
        'winner':mm.winner
    }
    file_name=f'output/{playerRed.name}_{playerBlue.name}_{board.name}_{i}.json'
    print(file_name)
    with open(file_name, 'w') as json_file:
        json.dump(save, json_file)
        #salva le cose su disco!

In [ ]:
agents = [GreedyAgent, SimpleMLAgent, RandomAgent]
scenarios = [scenarioTest1v1, scenarioTest2v2, scenarioJunction]
args=[]
for i in range(0, 100):
    for s in scenarios:
        for agentRed, agentBlue in product(agents, agents):
            args.append((i,s,agentRed,agentBlue))

In [ ]:
with Pool(3) as p:
    p.map(game, args)